In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from statsmodels import robust
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import os
import matplotlib.cm as cm

# run pca and visualize

import scipy

colors = [
'black','blue','red','green','cyan','magenta','brown','pink',
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']



Autosaving every 180 seconds


In [2]:

# visualize recomputed templates over time;
def binary_reader_waveforms(filename, n_channels, n_times, spikes, channels=None, data_type='float32'):
    ''' Reader for loading raw binaries
    
        standardized_filename:  name of file contianing the raw binary
        n_channels:  number of channels in the raw binary recording 
        n_times:  length of waveform 
        spikes: 1D array containing spike times in sample rate of raw data
        channels: load specific channels only
        data_type: float32 for standardized data
    
    '''

    # ***** LOAD RAW RECORDING *****
    if channels is None:
        wfs = np.zeros((spikes.shape[0], n_times, n_channels), data_type)
    else:
        wfs = np.zeros((spikes.shape[0], n_times, channels.shape[0]), data_type)
    if data_type =='float32':
        data_len = 4
    else:
        data_len = 2

    with open(filename, "rb") as fin:
        for ctr,s in enumerate(spikes):
            #print (ctr,s)
            # index into binary file: time steps * 4  4byte floats * n_channels
            fin.seek(s * data_len * n_channels, os.SEEK_SET)
            wfs[ctr] = np.fromfile(
                fin,
                dtype=data_type,
                count=(n_times * n_channels)).reshape(n_times, n_channels)[:,channels]
    fin.close()
    return wfs

In [3]:
# YASS DATASETS
# load raw data and spike train
root_dir = '/media/cat/12TB/insync_cm3746/Data/patrick/'

temps = np.load('/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/liam/512channels/2009-04-13-5_120mins/tmp/output/templates/templates_0sec.npy')
print ("templates: ", temps.shape)

spike_train = np.load('/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/liam/512channels/2009-04-13-5_120mins/tmp/output/spike_train.npy')
print ("spike_train: ", spike_train.shape)

geom = np.loadtxt('/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/liam/512channels/2009-04-13-5_120mins/geom.txt')
print ("geom: ", geom.shape)

data = np.fromfile('/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/liam/512channels/2009-04-13-5_120mins/tmp/preprocess/standardized.bin','float32')
print (data.shape)

n_chan = 512
data2D = data.reshape(-1,n_chan)
print (data2D.shape)



templates:  (2413, 101, 512)
spike_train:  (94794370, 2)
geom:  (512, 2)
(3072000000,)
(6000000, 512)


In [ ]:
# insert 7 channel templates + 

In [9]:
# SAVE NEURONS ON MAX CHAN AND IMMEDIATE NEGIHBROU CHANS
ptps = temps.ptp(1).max(1)
max_chans = temps.ptp(1).argmax(1)

root_dir = '/media/cat/12TB/insync_cm3746/Data/patrick/'

ctr=0
for chan in range(n_chan)[:10]:
    idx2 = np.where(max_chans==chan)[0]
    ptps_local = ptps[idx2]

    locs = geom[chan]
    
    thresh=70
    
    chans_neighbour = []
    for k in range(n_chan):
        dist = np.linalg.norm(locs-geom[k])
        if dist < thresh:
            chans_neighbour.append(k)

            # all chans within neighbourhood
    chans_neighbour = np.unique(chans_neighbour)
    
    print ("# chans enighour: ", chans_neighbour.shape)
    #continue
    #idx3 = np.where(ptps_local>8.0)[0]
    
    
    if idx2.shape[0]>5:
        
        # save geometry of the data
        np.savetxt(root_dir +'/chan_'+str(ctr)+'_geom.txt',
               geom[chans_neighbour])
        
        
        # save this channel and the data
        np.save(root_dir+'/chan_'+str(ctr)+'_data_7chan.npy',
                data2D[:,chans_neighbour])
        
        # save templates
        temps_out=[]
        for ch in chans_neighbour:
            idx = np.where(max_chans==ch)[0]
            #print (temps.shape, temps[idx].shape, temps[idx][:,:,chans_neighbour].shape)
            temps_out.extend(temps[idx][:,:,chans_neighbour])  #Make sure you save max channel tempalte NOT 
            
        temps_out = np.array(temps_out)[:,20:95]
        print (" templates saved: ", temps_out.shape)
            
        np.save(root_dir+'/chan_'+str(ctr)+'_templates.npy',
                temps_out)
        
        # save spiketrains
        times_out= []
        for ch in chans_neighbour:
            #idx = np.where
            ids = np.where(max_chans==ch)[0]
            for id_ in ids:
                idx4 = np.where(spike_train[:,1]==id_)[0]
                times = spike_train[idx4,0]
                idx5 = np.where(times<(5*60*20000))[0]
                times_out.append(times[idx5])
        
        np.save(root_dir+'/chan_'+str(ctr)+'_times.npy',
                times_out)
        #break    
        ctr+=1

# chans enighour:  (7,)
# chans enighour:  (7,)
 templates saved:  (29, 75, 7)


/home/cat/anaconda3/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


# chans enighour:  (7,)
# chans enighour:  (7,)
# chans enighour:  (7,)
# chans enighour:  (7,)
# chans enighour:  (7,)
# chans enighour:  (5,)
# chans enighour:  (7,)
 templates saved:  (22, 75, 7)
# chans enighour:  (7,)


In [ ]:
# grab all clustered spikes on a single channel:
channel = 10

ptps = temps.ptp(1).max(1)
max_chans = temps.ptp(1).argmax(1)

root_dir = '/media/cat/12TB/insync_cm3746/Data/patrick/'

idx2 = np.where(max_chans==chan)[0]
ptps_local = ptps[idx2]

locs = geom[chan]

thresh=70

chans_neighbour = []
for k in range(n_chan):
    dist = np.linalg.norm(locs-geom[k])
    if dist < thresh:
        chans_neighbour.append(k)

        # all chans within neighbourhood
chans_neighbour = np.unique(chans_neighbour)

print ("# chans enighour: ", chans_neighbour.shape)
#continue
#idx3 = np.where(ptps_local>8.0)[0]


if idx2.shape[0]>5:

    # save geometry of the data
    np.savetxt(root_dir +'/chan_'+str(ctr)+'_geom.txt',
           geom[chans_neighbour])


    # save this channel and the data
    np.save(root_dir+'/chan_'+str(ctr)+'_data_7chan.npy',
            data2D[:,chans_neighbour])

    # save templates
    temps_out=[]
    for ch in chans_neighbour:
        idx = np.where(max_chans==ch)[0]
        #print (temps.shape, temps[idx].shape, temps[idx][:,:,chans_neighbour].shape)
        temps_out.extend(temps[idx][:,:,chans_neighbour])  #Make sure you save max channel tempalte NOT 

    temps_out = np.array(temps_out)[:,20:95]
    print (" templates saved: ", temps_out.shape)

    np.save(root_dir+'/chan_'+str(ctr)+'_templates.npy',
            temps_out)

    # save spiketrains
    times_out= []
    for ch in chans_neighbour:
        #idx = np.where
        ids = np.where(max_chans==ch)[0]
        for id_ in ids:
            idx4 = np.where(spike_train[:,1]==id_)[0]
            times = spike_train[idx4,0]
            idx5 = np.where(times<(5*60*20000))[0]
            times_out.append(times[idx5])

    np.save(root_dir+'/chan_'+str(ctr)+'_times.npy',
            times_out)
    #break    
    ctr+=1

In [ ]:
# grab all clustered spikes on a single channel:
channel = 10


